In [2]:
import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks/fedml')


In [1]:
# -*- coding: utf-8 -*-
import csv
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torchvision import datasets, transforms, models
import copy
import argparse
import torch.nn.functional as F
import random
from torch.utils.data import DataLoader, random_split ,TensorDataset
import resample_data
import choose_models
import tools
import fedserver
import fedclient
import math


In [ ]:
def main_fedfastslowmon(model_type,learning_rate, momentum, nesterov ,num_rounds, local_round, num_clients ,batch_size, loss_function,dataset):
    #tools.cleanexcel('/content/drive/MyDrive/Colab Notebooks/fedml/result_N_T.xlsx','fastslowmo_'+str(num_clients)+'_'+str(local_round))
    #tools.cleanexcel('result_N_T.xlsx','fastslowmo_'+str(num_clients)+'_'+str(local_round))
    device=tools.choose_device()
    model=choose_models.select_model(model_type)
    train_dataset, test_dataset= choose_models.select_dataset(dataset)
    model.to(device)
    # # Create data loaders for each client
    client_datasets= resample_data.data_distribution_0_1(train_dataset,len(train_dataset.classes), num_clients)
    train_loaders = []
    for i in range(num_clients):
        train_loader = torch.utils.data.DataLoader(dataset=client_datasets[i], batch_size=batch_size, shuffle=True)
        train_loaders.append(train_loader)
    test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

    #server and client
    model=copy.deepcopy(model)
    model.to(device)
    server=fedserver.Server(model, learning_rate, momentum, nesterov, device)
    server.global_optimizer=optim.SGD(server.global_model.parameters(), lr=learning_rate, momentum=0.5, nesterov=True)
    server.loss_function(loss_function)
    client_name_list=tools.set_client(num_clients)
    clients=[]
    for i in range(len(client_name_list)):
         clients.append(fedclient.Client(id= client_name_list[i],data=train_loaders[i],local_round=local_round, device=device))
         clients[i].local_model=copy.deepcopy(model)
         clients[i].local_optimizer= optim.SGD(clients[i].local_model.parameters(), lr=learning_rate, momentum=0.5, nesterov=True)
         server.register(clients[i])


    for i in range(num_rounds):
          print(i)
          server.current_global_round=i
          for i in range(len(clients)):
               server.local_train(client_name_list[i])
          server.aggregate_fastslowmon()
          for i in range(len(clients)):
               server.download_model_faseslowmon(client_name_list[i])
               
          server.loss=server.get_loss(server.global_model,test_loader)
          server.acc=server.get_accuracy(server.global_model,test_loader)
          result=server.result()
          sheetname='fastslowmo_'+str(num_clients)+'_'+str(local_round)
          #tools.save2excel('/content/drive/MyDrive/Colab Notebooks/fedml/result_N_T.xlsx','fastslowmo_'+str(num_clients)+'_'+str(local_round),result)
          tools.save2excel('result_N_T.xlsx',sheetname,result)


T=[10,20,40,80,160]
N=[2,4,8,16]
for i in range(5):
    for j in range(4):
            main_fedfastslowmon('cnn',0.01,0.5,True,math.ceil(1000/T[i]),T[i],N[j],64,'nll_loss','mnist')

In [4]:
def main_fedslowmon(model_type,learning_rate, momentum, nesterov ,num_rounds, local_round, num_clients ,batch_size, loss_function,dataset):
    device=tools.choose_device()
    model=choose_models.select_model(model_type)
    train_dataset, test_dataset= choose_models.select_dataset(dataset)
    model.to(device)
    # # Create data loaders for each client
    client_datasets= resample_data.data_distribution_0_1(train_dataset,len(train_dataset.classes), num_clients)
    train_loaders = []
    for i in range(num_clients):
        train_loader = torch.utils.data.DataLoader(dataset=client_datasets[i], batch_size=batch_size, shuffle=True)
        train_loaders.append(train_loader)
    test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

    #server and client
    model=copy.deepcopy(model)
    model.to(device)
    server=fedserver.Server(model, learning_rate, momentum, nesterov, device)
    server.global_optimizer=optim.SGD(server.global_model.parameters(), lr=learning_rate)
    server.loss_function(loss_function)
    client_name_list=tools.set_client(num_clients)
    clients=[]
    for i in range(len(client_name_list)):
         clients.append(fedclient.Client(id= client_name_list[i],data=train_loaders[i],local_round=local_round, device=device))
         clients[i].local_model=copy.deepcopy(model)
         clients[i].local_optimizer= optim.SGD(clients[i].local_model.parameters(), lr=learning_rate)
         server.register(clients[i])
    results=[]
    for i in range(num_rounds):
          print(i)
          server.current_global_round=i
          for i in range(len(client_name_list)):
               server.local_train(client_name_list[i])
          server.aggregate_slowmon()
          for i in range(len(client_name_list)):
               server.download_model_slowmon(client_name_list[i]) 

          server.loss=server.get_loss(server.global_model,test_loader)
          server.acc=server.get_accuracy(server.global_model,test_loader)
          print(server.acc)
          result=server.result()
          results.append(result)
    sheetname='slowmo_'+str(num_clients)+'_'+str(local_round)
    tools.save2excel_batch('result_all.xlsx',sheetname,results)
#tools.cleanexcel('result_all.xlsx','mnist_linear_slowmon')
T=[10,20,40,80,160]
N=[2,4,8,16]
for i in range(5):
    for j in range(4):
        if i == 2 or j== 0:
            main_fedslowmon('cnn',0.01,0,False,math.ceil(1000/T[i]),T[i],N[j],64,'nll_loss','mnist')


0


/Users/jinyiw/opt/anaconda3/lib/python3.9/site-packages/torch/nn/functional.py:1331: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)


20.47
1
50.47
2
68.15
3
75.81
4
81.5
5
82.19
6
85.76
7
85.3
8
86.8
9
88.58
10
89.19
11
88.42
12
89.19
13
88.62
14
89.44
15
90.46
16
91.06
17
90.98
18
90.96
19
91.32
20
91.32
21
91.96
22
91.94
23
92.13
24
91.66
25
91.61
26
92.16
27
91.98
28
92.61
29
92.78
30
92.98
31
93.0
32
93.12
33
92.87
34
93.19
35
92.95
36
93.24
37
93.63
38
93.38
39
93.52
40
93.8
41
93.91
42
94.09
43
93.91
44
93.87
45
93.86
46
94.03
47
93.9
48
93.83
49
94.32
50
94.29
51
94.12
52
94.31
53
94.33
54
94.5
55
94.51
56
94.48
57
94.29
58
94.7
59
94.89
60
94.67
61
94.82
62
94.79
63
94.44
64
94.89
65
94.71
66
94.53
67
94.84
68
94.98
69
94.98
70
94.84
71
95.03
72
95.41
73
95.28
74
94.96
75
95.02
76
95.33
77
94.81
78
95.38
79
95.07
80
95.22
81
95.52
82
95.25
83
95.01
84
95.26
85
95.46
86
95.36
87
95.05
88
95.25
89
95.31
90
95.62
91
95.42
92
95.36
93
95.48
94
95.56
95
95.6
96
95.48
97
95.73
98
95.62
99
95.71
0
69.01
1
74.39
2
79.23
3
85.89
4
88.23
5
88.9
6
89.65
7
89.84
8
90.37
9
90.84
10
91.26
11
91.64
12
91.88
13
92.18
14
92.

In [2]:
def main_fedmon(model_type,learning_rate, momentum, nesterov ,num_rounds, local_round, num_clients ,batch_size, loss_function,dataset):
    device=tools.choose_device()
    model=choose_models.select_model(model_type)
    train_dataset, test_dataset= choose_models.select_dataset(dataset)
    model.to(device)
    # # Create data loaders for each client
    client_datasets= resample_data.data_distribution_0_1(train_dataset,len(train_dataset.classes), num_clients)
    train_loaders = []
    for i in range(num_clients):
        train_loader = torch.utils.data.DataLoader(dataset=client_datasets[i], batch_size=batch_size, shuffle=True)
        train_loaders.append(train_loader)
    test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

    #server and client
    model=copy.deepcopy(model)
    model.to(device)
    server=fedserver.Server(model, learning_rate, momentum, nesterov, device)
    server.global_optimizer=optim.SGD(server.global_model.parameters(), lr=learning_rate)
    server.loss_function(loss_function)
    client_name_list=tools.set_client(num_clients)
    clients=[]
    for i in range(len(client_name_list)):
         clients.append(fedclient.Client(id= client_name_list[i],data=train_loaders[i],local_round=local_round, device=device))
         clients[i].local_model=copy.deepcopy(model)
         server.register(clients[i])

    results=[]
    for i in range(num_rounds):
          for i in range(len(client_name_list)): 
            server.download_model_mon(client_name_list[i])   
            server.current_global_round=i
            server.local_train(client_name_list[i])
          server.aggregate_fedmon()

          server.loss=server.get_loss(server.global_model,test_loader)
          server.acc=server.get_accuracy(server.global_model,test_loader)
          result=server.result()
          results.append(result)
    sheetname='fedmo_'+str(num_clients)+'_'+str(local_round)
    tools.save2excel_batch('result_N_T.xlsx',sheetname,results)
T=[10,20,40,80,160]
N=[2,4,8,16]
for i in range(5):
    for j in range(4):
        main_fedmon('cnn',0.01,0,False,math.ceil(1000/T[i]),T[i],N[j],64,'nll_loss','mnist')


/Users/jinyiw/opt/anaconda3/lib/python3.9/site-packages/torch/nn/functional.py:1331: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)


In [6]:
def main_nag(model_type,learning_rate, momentum, nesterov ,num_rounds, local_round, num_clients ,batch_size, loss_function,dataset):
    device=tools.choose_device()
    model=choose_models.select_model(model_type)
    train_dataset, test_dataset= choose_models.select_dataset(dataset)
    model.to(device)
    # # Create data loaders for each client
    client_datasets= resample_data.data_distribution_0_1(train_dataset,len(train_dataset.classes), num_clients)
    train_loaders = []
    for i in range(num_clients):
        train_loader = torch.utils.data.DataLoader(dataset=client_datasets[i], batch_size=batch_size, shuffle=True)
        train_loaders.append(train_loader)
    test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

    #server and client
    model=copy.deepcopy(model)
    model.to(device)
    server=fedserver.Server(model, learning_rate, momentum, nesterov, device)
    server.global_optimizer=optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum, nesterov= nesterov)
    server.loss_function(loss_function)
    client_name_list=tools.set_client(num_clients)
    clients=[]
    for i in range(len(client_name_list)):
         clients.append(fedclient.Client(id= client_name_list[i],data=train_loaders[i],local_round=local_round, device=device))
         server.register(clients[i])
    results=[]
    for i in range(num_rounds):
          for idx in range(len(client_name_list)):
              server.download_model(client_name_list[idx])
          server.current_global_round=i
          for idx in range(len(client_name_list)):

               server.local_train(client_name_list[idx])
          server.aggregate_fednag()
          server.loss=server.get_loss(server.global_model,test_loader)
          server.acc=server.get_accuracy(server.global_model,test_loader)
          result=server.result()
          results.append(result)
    if momentum!=0:
        sheetname='fednag_'+str(num_clients)+'_'+str(local_round)
    if momentum==0:
        sheetname='fedavg_'+str(num_clients)+'_'+str(local_round)
    tools.save2excel_batch('result_N_T.xlsx',sheetname,results)
T=[10,20,40,80,160]
N=[2,4,8,16]
for i in range(5):
    for j in range(4):
        main_nag('cnn',0.01,0.5,True,math.ceil(1000/T[i]),T[i],N[j],64,'nll_loss','mnist')



/Users/jinyiw/opt/anaconda3/lib/python3.9/site-packages/torch/nn/functional.py:1331: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)


In [7]:
for i in range(5):
    for j in range(4):
        main_nag('cnn',0.01,0,False,math.ceil(1000/T[i]),T[i],N[j],64,'nll_loss','mnist')